In [ ]:
import pandas as pd
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


타이타닉 탑승객의 생존 여부를 예측하는 모형 제작

# KNN

##데이터 준비

In [ ]:
df = sns.load_dataset("titanic") #데이터셋을 가져와 데이터프레임으로 변환
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


##데이터 탐색

In [ ]:
df.info() #데이터의 자료형, 개수 등 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [ ]:
#deck 열의 유효한 값이 203개 - 제거 
#embark_town(승선도시)열은 embarked 열과 동일한 의미 - 중복을 없애기 위해 제거

In [ ]:
rdf = df.drop(["deck", "embark_town"], axis = 1)
rdf.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

In [ ]:
rdf = rdf.dropna(subset = ["age"], how = "any", axis = 0) #나이 데이터가 없는 모든 행 삭제

In [ ]:
#embarked 열의 누락 데이터는 2개
#탑승한 승객이 가장 많은 도시명 찾기 .value_counts(), .idxmax()
most_freq = rdf["embarked"].value_counts(dropna = True).idxmax()
print("탑승객이 가장 많은 도시: ", most_freq)
print()

rdf.describe(include = "all") #embarked 열의 최빈값 확인 가능(방법 2)

탑승객이 가장 많은 도시:  S



,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,712,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,554,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rdf["embarked"].fillna(most_freq, inplace = True) #누락 데이터 채워주기

##속성 선택

In [ ]:
ndf = rdf[["survived", "pclass", "sex", "age", "sibsp", "parch", "embarked"]]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [ ]:
#KNN 모형 적용을 위해서
#sex 열과 embarked 열의 범주형 데이터를 숫자형으로 변환
#해당 과정을 더미 변수를 만든다는 의미로 '원-핫-인코딩'이라고 부름

In [ ]:
#원핫인코딩
onehot_sex = pd.get_dummies(ndf["sex"]) #더미변수 열 생성(sex 기준, male/female)
ndf = pd.concat([ndf, onehot_sex], axis = 1) #.concat() 함수로 더미변수 열을 기존 데이터프레임에 연결 

onehot_embarked = pd.get_dummies(ndf["embarked"], prefix = "town")
ndf = pd.concat([ndf, onehot_embarked], axis = 1)

In [ ]:
#기존의 범주형 열 sex, embarked 삭제
ndf.drop(["sex", "embarked"], axis = 1, inplace = True)

In [ ]:
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


##데이터 분할

In [ ]:
X = ndf[["pclass", "age", "sibsp", "parch", "female", "male", "town_C", "town_Q", "town_S"]]
y = ndf["survived"]

In [ ]:
from sklearn import preprocessing 
X = preprocessing.StandardScaler().fit(X).transform(X) #설명변수 데이터 정규화(normalization) - 상대적 크기 차이를 없애기 위함

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 10)

##모형 학습 및 검증

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5) #모형 객체 생성(k = 5 설정)

In [ ]:
knn.fit(X_train, y_train) #모형 학습
y_hat = knn.predict(X_test) #test data로 예측

In [ ]:
#모델 성능 평가 - confusion matrix
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat)
knn_matrix

array([[109,  16],
       [ 25,  65]])

In [ ]:
#모델 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, y_hat)
print(knn_report)

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       125
           1       0.80      0.72      0.76        90

    accuracy                           0.81       215
   macro avg       0.81      0.80      0.80       215
weighted avg       0.81      0.81      0.81       215



#SVM

In [ ]:
from sklearn import svm
svm_model = svm.SVC(kernel = "rbf") #모형 객체 생성
svm_model.fit(X_train, y_train) #모형 학습
y_hat = svm_model.predict(X_test) #테스트 데이터 예측

In [ ]:
#모형 성능 평가 - confusion matrix
svm_matrix = metrics.confusion_matrix(y_test, y_hat) 
svm_matrix

array([[120,   5],
       [ 35,  55]])

In [ ]:
#모형 성능 평가 - 평가 지표
svm_report = metrics.classification_report(y_test, y_hat)
print(svm_report)

              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215

